In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from typing import Type
from simulator import CovertGenerator, DarkGenerator
import numpy as np
import pandas as pd
import json

NameError: name 'ba_generator' is not defined

In [63]:
# 

## Generator & Reader

In [4]:
def ba_generator(min_n: int=20, max_n: int=30, m: int=2) -> Type[nx.classes.graph.Graph]:
    n = np.random.randint(max_n - min_n + 1) + min_n
    return nx.barabasi_albert_graph(n ,m)

def covert_generator(min_n: int=20, max_n: int=30, density: int=0.1):
    generator = CovertGenerator(min_n, max_n, density)
    generator.simulate()
    return generator.G

def dark_generator(min_n: int=20, max_n: int=30, density: int=0.1):
    generator = DarkGenerator(min_n, max_n, density)
    generator.simulate()
    return generator.G

def write_gml(G: Type[nx.classes.graph.Graph], data_dir: str, file_name: str) -> None:
    nx.write_gml(G, data_dir+file_name)
    
def read_gml(data_dir: str, file_name: str) -> Type[nx.classes.graph.Graph]:
    G = nx.read_gml(data_dir + file_name)
    return nx.relabel_nodes(G, lambda x: int(x))

def get_real_graph(data_dir: str, file_name: str) -> Type[nx.classes.graph.Graph]:
    return read_gml(data_dir, file_name)

def getRobustness(g: Type[nx.classes.graph.Graph], sol: int):
    G = g.copy()
    G_size = G.number_of_nodes()
    GCCsize = len(max(nx.connected_components(G), key=len))
    
    removed = [n for n in G.neighbors(int(sol))] + [int(sol)]
    for n in removed:
        G.remove_node(n)
        
    newGCCsize = len(max(nx.connected_components(G), key=len))

    return (GCCsize - newGCCsize) / (G_size * G_size)

In [5]:
def HXA(g: Type[nx.classes.graph.Graph], method: str) -> (list, list):
    # 'HDA', 'HBA', 'HCA', 'HPRA'
    sol = []
    reward = []
    G = g.copy()
    while (nx.number_of_edges(G)>0):
        if method == 'HDA':
            dc = nx.degree_centrality(G)
        elif method == 'HBA':
            dc = nx.betweenness_centrality(G)
        elif method == 'HCA':
            dc = nx.closeness_centrality(G)
        elif method == 'HPRA':
            dc = nx.pagerank(G)
        keys = list(dc.keys())
        values = list(dc.values())
        maxTag = np.argmax(values)
        node = keys[maxTag]
        
        reward.append(getRobustness(G, int(node)))
        
        sol.append(int(node))
        removed = [n for n in G.neighbors(int(node))] + [int(node)]
        for n in removed:
            G.remove_node(n)

    return sol, reward

In [14]:
ls

Untitled.ipynb         dark/                  simulator.py
__pycache__/           graph_generator.ipynb  synthetic/
ba/                    graph_generator.py     test_graph/
covert/                real/


## BA & Covert & Dark 

In [15]:
# iters = 100
# for i in range(iters):
#     G = ba_generator(min_n=200, max_n=200, m=1)
#     write_gml(G, data_dir="./ba/", file_name=f"g_{i}")

In [5]:
# iters = 100
# for i in range(iters):
#     G = dark_generator(min_n=200, max_n=200, density=0.01)
#     write_gml(G, data_dir="./dark/", file_name=f"g_{i}")

In [33]:
# iters = 100
# for i in range(iters): 
#     G = covert_generator(min_n=200, max_n=200, density=0.01)
#     write_gml(G, data_dir="./covert/", file_name=f"g_{i}")

## Heuristic based

In [6]:
# gcc = 
for i in range(100):
    hxa_dct = dict()
    reward_dct = dict()
    g = read_gml(data_dir="./dark/", file_name=f"g_{i}")

    for method in ['HDA', 'HBA', 'HCA', 'HPRA']:
        sol, reward = HXA(g=g, method=method)
        hxa_dct[method] = sol
        reward_dct[method] = np.cumsum(reward).tolist()
        
        break
#     with open(f"./dark/node_hist/g_{i}.json", "w") as json_file:
#         json.dump(hxa_dct, json_file) 
#     with open(f"./dark/reward_hist/g_{i}.json", "w") as json_file:
#         json.dump(reward_dct, json_file)
        

In [7]:
for i in range(100):
    finder_dct = dict()
    reward_lst = []
    g = read_gml(data_dir="./dark/", file_name=f"g_{i}")
    df = pd.read_csv(f"./dark/finder_node_hist/g_{i}.txt", header=None).rename(columns={0: "round", 1: "node"})
    
    for _, (round, node) in df.iterrows():
        try:
            reward_lst.append(getRobustness(g, int(node)))
            removed = [n for n in g.neighbors(int(node))] + [int(node)]
            for n in removed:
                g.remove_node(n)
        except:
            pass
        print("edge", nx.number_of_edges(g), nx.number_of_nodes(g))
        print("gcc", len(max(nx.connected_components(g), key=len)))
    with open(f"./dark/finder_reward_hist/g_{i}.json", "w") as json_file:
        json.dump(np.cumsum(reward_lst).tolist(), json_file)

edge 171 198
gcc 46
edge 165 196
gcc 43
edge 165 195
gcc 43
edge 141 193
gcc 41
edge 127 190
gcc 38
edge 105 188
gcc 36
edge 101 186
gcc 34
edge 85 183
gcc 30
edge 85 182
gcc 30
edge 81 180
gcc 28
edge 81 180
gcc 28
edge 65 178
gcc 26
edge 54 176
gcc 24
edge 54 176
gcc 24
edge 43 173
gcc 21
edge 32 170
gcc 17
edge 32 169
gcc 17
edge 32 169
gcc 17
edge 25 166
gcc 14
edge 25 166
gcc 14
edge 25 166
gcc 14
edge 25 166
gcc 14
edge 17 163
gcc 11
edge 17 163
gcc 11
edge 17 163
gcc 11
edge 4 160
gcc 3
edge 4 160
gcc 3
edge 2 158
gcc 2
edge 2 158
gcc 2
edge 2 157
gcc 2
edge 2 157
gcc 2
edge 1 155
gcc 2
edge 1 155
gcc 2
edge 1 155
gcc 2
edge 1 155
gcc 2
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 260 198
gcc 46
edge 251 196
gcc 44
edge 235 194
gcc 42
edge 223 192
gcc 40
edge 208 190
gcc 38
edge 191 188
gcc 36
edge 168 186
gcc 34
edge 149 184
gcc 32
edge 127 182
gcc 3

edge 171 198
gcc 41
edge 156 195
gcc 38
edge 142 193
gcc 36
edge 136 191
gcc 34
edge 126 189
gcc 32
edge 115 187
gcc 30
edge 115 187
gcc 30
edge 101 183
gcc 26
edge 86 181
gcc 24
edge 70 178
gcc 21
edge 44 174
gcc 17
edge 44 174
gcc 17
edge 31 172
gcc 14
edge 28 170
gcc 12
edge 28 170
gcc 12
edge 28 170
gcc 12
edge 28 170
gcc 12
edge 14 167
gcc 9
edge 4 164
gcc 5
edge 4 164
gcc 5
edge 4 164
gcc 5
edge 4 164
gcc 5
edge 4 164
gcc 5
edge 4 164
gcc 5
edge 4 164
gcc 5
edge 4 164
gcc 5
edge 4 163
gcc 5
edge 2 161
gcc 3
edge 2 160
gcc 3
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 158
gcc 1
edge 0 157
gcc 1
edge 0 157
gcc 1
edge 161 198
gcc 38
edge 149 196
gcc 36
edge 145 194
gcc 34
edge 128 192
gcc 32
edge 109 190
gcc 29
edge 94 187
gcc 26
edge 70 183
gcc 22
edge 53 180
gcc 19
edge 53 179
gcc 19
edge 53 179
gcc 19
edge 53 179
gcc 19
edge 21 173
gcc 13
edge 21 17

edge 242 198
gcc 50
edge 226 196
gcc 48
edge 208 194
gcc 45
edge 208 193
gcc 45
edge 189 191
gcc 43
edge 169 189
gcc 41
edge 162 187
gcc 39
edge 158 185
gcc 37
edge 136 183
gcc 35
edge 116 179
gcc 31
edge 116 179
gcc 31
edge 80 175
gcc 27
edge 67 172
gcc 24
edge 50 169
gcc 21
edge 50 169
gcc 21
edge 41 166
gcc 18
edge 41 166
gcc 18
edge 41 166
gcc 18
edge 33 164
gcc 15
edge 33 164
gcc 15
edge 22 161
gcc 12
edge 12 158
gcc 8
edge 6 156
gcc 6
edge 6 155
gcc 6
edge 6 155
gcc 6
edge 3 153
gcc 4
edge 3 152
gcc 4
edge 3 152
gcc 4
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 219 198
gcc 42
edge 206 196
gcc 40
edge 188 194
gcc 38
edge 159 191
gcc 35

edge 26 166
gcc 15
edge 21 164
gcc 13
edge 13 161
gcc 10
edge 13 161
gcc 10
edge 13 160
gcc 10
edge 9 158
gcc 7
edge 9 158
gcc 7
edge 5 156
gcc 5
edge 2 154
gcc 3
edge 0 152
gcc 1
edge 0 152
gcc 1
edge 0 151
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 198 198
gcc 44
edge 198 197
gcc 44
edge 181 195
gcc 42
edge 178 193
gcc 40
edge 145 190
gcc 37
edge 134 188
gcc 35
edge 108 185
gcc 31
edge 85 181
gcc 27
edge 70 178
gcc 24
edge 70 177
gcc 24
edge 46 174
gcc 20
edge 24 170
gcc 14
edge 16 168
gcc 11
edge 16 168
gcc 11
edge 16 168
gcc 11
edge 16 167
gcc 11
edge 16 166
gcc 11
edge 10 164
gcc 8
edge 10 163
gcc 8
edge 10 163


edge 243 196
gcc 49
edge 223 194
gcc 47
edge 223 193
gcc 47
edge 217 191
gcc 45
edge 190 189
gcc 43
edge 168 186
gcc 40
edge 163 184
gcc 38
edge 145 181
gcc 34
edge 129 179
gcc 32
edge 107 176
gcc 29
edge 99 174
gcc 27
edge 99 173
gcc 27
edge 85 171
gcc 25
edge 85 171
gcc 25
edge 72 168
gcc 22
edge 46 165
gcc 19
edge 44 163
gcc 17
edge 44 163
gcc 17
edge 44 163
gcc 17
edge 38 161
gcc 15
edge 30 159
gcc 13
edge 30 159
gcc 13
edge 22 157
gcc 10
edge 22 156
gcc 10
edge 22 156
gcc 10
edge 8 153
gcc 7
edge 8 153
gcc 7
edge 8 153
gcc 7
edge 5 151
gcc 5
edge 5 151
gcc 5
edge 5 151
gcc 5
edge 5 151
gcc 5
edge 5 151
gcc 5
edge 5 151
gcc 5
edge 1 149
gcc 2
edge 1 149
gcc 2
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 144 198
gcc 42
edge 130 196
gcc 39
edge 130 195
gcc

edge 0 147
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 0 145
gcc 1
edge 253 198
gcc 49
edge 236 196
gcc 47
edge 215 194
gcc 45
edge 209 192
gcc 43
edge 194 189
gcc 40
edge 175 187
gcc 38
edge 152 185
gcc 36
edge 131 182
gcc 32
edge 131 181
gcc 32
edge 124 179
gcc 30
edge 111 177
gcc 28
edge 86 174
gcc 25
edge 63 171
gcc 22
edge 38 167
gcc 17
edge 38 167
gcc 17
edge 38 166
gcc 17
edge 18 161
gcc 12
edge 12 158
gcc 9
edge 6 155
gcc 6
edge 6 155
gcc 6
edge 6 155
gcc 6
edge 3 153
gcc 4
edge 3 153
gcc 4
edge 3 153
gcc 4
edge 3 153
gcc 4
edge 3 153
gcc 4
edge 3 153
gcc 4
edge 3 153
gcc 4
edge 3 153
gcc 4
edge 3 153
gcc 4
edge 1 151
gcc 2
edge 0 149
gcc 1
ed

gcc 34
edge 118 176
gcc 32
edge 109 174
gcc 30
edge 109 173
gcc 30
edge 105 171
gcc 28
edge 84 168
gcc 24
edge 68 165
gcc 21
edge 68 165
gcc 21
edge 56 162
gcc 18
edge 56 161
gcc 18
edge 37 158
gcc 15
edge 37 158
gcc 15
edge 19 155
gcc 12
edge 10 152
gcc 9
edge 8 150
gcc 7
edge 8 150
gcc 7
edge 8 150
gcc 7
edge 8 150
gcc 7
edge 8 150
gcc 7
edge 6 148
gcc 5
edge 6 148
gcc 5
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 144
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 0 143
gcc 1
edge 126 198
gcc 38
edge 126 197
gcc 38
edge 123 195
gcc 36
edge 111 193
gcc 34
edge 102 191
gcc 32
edge 88 188
gcc 29
edge 75 186
gcc 27
edge 75 186
gcc 27
edge 62 184
gcc 25
edge 54 181
gcc 22
edge 50 179
gcc 20
ed

edge 143 178
gcc 30
edge 134 176
gcc 28
edge 134 175
gcc 28
edge 98 171
gcc 24
edge 67 167
gcc 20
edge 67 167
gcc 20
edge 55 165
gcc 18
edge 31 161
gcc 14
edge 31 161
gcc 14
edge 9 156
gcc 8
edge 6 154
gcc 6
edge 6 154
gcc 6
edge 6 154
gcc 6
edge 2 152
gcc 3
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 150
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 149
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 148
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 0 147
gcc 1
edge 233 198
gcc 47
edge 222 196
gcc 45
edge 222 195
gcc 45
edge 199 192
gcc 42
edge 185 190
gcc 40
edge 149 187
gcc 36
edge 134 185
gcc 34
edge 109 182
gcc 31
edge 109 181
gcc 31
edge 100 179
gcc 29
edge 87 177
gcc 27
edge 74 175
gcc 25
edge 52 17

edge 121 184
gcc 38
edge 103 181
gcc 35
edge 93 179
gcc 33
edge 83 177
gcc 31
edge 63 174
gcc 28
edge 58 172
gcc 26
edge 51 169
gcc 23
edge 48 167
gcc 21
edge 48 167
gcc 21
edge 48 167
gcc 21
edge 27 163
gcc 17
edge 27 163
gcc 17
edge 27 163
gcc 17
edge 25 161
gcc 15
edge 14 158
gcc 10
edge 14 158
gcc 10
edge 10 156
gcc 8
edge 10 156
gcc 8
edge 10 155
gcc 8
edge 6 153
gcc 6
edge 4 151
gcc 4
edge 4 151
gcc 4
edge 4 151
gcc 4
edge 1 149
gcc 2
edge 1 148
gcc 2
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 0 146
gcc 1
edge 194 198
gcc 46
edge 177 196
gcc 44
edge 162 194
gcc 42
edge 144 192
gcc 40
edge 125 190
gcc 38
edge 110 188
gcc 36
edge 93 186
gcc 34
edge 91 184
gcc 32
edge 6

edge 207 198
gcc 45
edge 192 196
gcc 43
edge 179 194
gcc 41
edge 160 192
gcc 39
edge 150 189
gcc 36
edge 132 187
gcc 34
edge 111 185
gcc 32
edge 88 182
gcc 29
edge 76 180
gcc 26
edge 60 177
gcc 23
edge 60 177
gcc 23
edge 49 174
gcc 20
edge 32 170
gcc 16
edge 32 170
gcc 16
edge 32 169
gcc 16
edge 32 169
gcc 16
edge 16 165
gcc 11
edge 9 163
gcc 9
edge 9 163
gcc 9
edge 9 163
gcc 9
edge 2 159
gcc 2
edge 2 158
gcc 2
edge 1 156
gcc 2
edge 1 156
gcc 2
edge 1 156
gcc 2
edge 1 155
gcc 2
edge 1 155
gcc 2
edge 1 155
gcc 2
edge 1 155
gcc 2
edge 1 155
gcc 2
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 0 153
gcc 1
edge 367 198
gcc 60
edge 339 196
gcc 58
edge 312 194
gcc 56
edge 288 192
gcc 54
edge 261 190
gcc 52
edge 235 188
gcc 49
edge 235 187
gcc 49
edge 215 184
gcc 46
edge 199 182
gcc 